In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import copy
import pickle
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Lambda, Multiply, Flatten, Reshape, Conv2D, MaxPooling2D, Add
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from matplotlib import rc, rcParams
from numpy.linalg import norm

In [ ]:
# Check if GPU is available
#tf.test.is_gpu_available()

In [ ]:
# Check tensorflow version
#tf.__version__

In [ ]:
# check state of GPUs
#!nvidia-smi

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
class Logger(object):
    """
    A helper class for debugging.
    """
    def __init__(self, output=False):
        self.output = output
    
    def log(self, message, value):
        if self.output:
            print(message, value)

## Section 0: NodePrune class contains methods for experiments

In [ ]:
class NodePrune(object):
    def flatten_mask(self, mask):
        """Returns the nodes of mask flattened
        Input:
        mask - the mask position, in dictionary form
        Output:
        mask_flatten - the mask position, flattened out in 1D array
        """
        mask_flatten = []
        for k, v in mask.items():
            current_mask = v.flatten()
            mask_flatten = np.hstack([mask_flatten, current_mask]) #append all mask. i.e make a total of 1D array of all mask

        return mask_flatten

    def reset_mask(self, mask):
        """Resets mask to initial start state of all ones"""
        for k, v in mask.items():
            mask[k] = np.ones_like(v)
        
        return mask

    def compare_mask(self, mask1, mask2):
        """ Compares how similar both masks (mask1, mask2) are and returns a percentage similarity """
        count = 0
        total_count = 0
        for k, v in mask1.items():
            count += np.sum(mask1[k] == mask2[k])
            total_count += len(mask1[k].ravel())
            
        return count/total_count

    def percent_mask(self, mask):
        """Returns the percentage of mask that contains 1s"""
        num_mask = 0
        total_mask = 0
        
        for v in mask.values():
            num_mask += np.sum(v)
            total_mask += len(v.ravel())
            
        return num_mask/total_mask

    def save_file(self, cache, name):
        """ Function which saves the cache """
        with open('Caches/'+name+'.p','wb') as outfile:
            pickle.dump(cache, outfile)
        
    def load_file(self, name):
        """ Function which loads the cache """
        with open('Caches/'+name+'.p','rb') as infile: 
            cache = pickle.load(infile)
        return cache

    def hoyer_square(self, weight):
        """
        Computes the square of hoyer regularization
        Args:
            weights: the weights in a given layer
        Returns: 
            hoyer_square: A scalar that penalizes the weights 
        """
        numerator = np.sum(np.abs(weight)) ** 2
        denominator = np.sum(weight ** 2)
        return numerator / denominator 

    def add_mask(self, mask, layer, layer_index, activation_array, initialize=True, in_out_layers=False):
        """Initialize mask for layers in MLPs.
        Args:
            layer: tf.keras layer
            mask: Dict - key is a layer, value is numpy array of containing 
                0s(nodes to be removed) or 1s(nodes to keep)
            activation_array: array of activations
            layer_index: Int - used track the position of each layer
            initialize: Boolean - If true initializes mask to all 1s, else uses the
                        mask corresponding to the current layer(i.e model) to drop 
                        nodes in the layer by doing element wise multiplication. 
            in_out_layers: Boolean - Indicates if mask should be created for input and 
                        output layers. Defaults to False as we are not dropping the
                        input and output nodes
        Returns:
            mask: Dict - key is a layer, value is numpy array of mask
            layer: updated tf.keras layer with mask
            activation_array: list of the post activations of each node in a layer
            layer_index: Int - unique number for naming layers 
        """

        layer_name = 'layer_' + str(layer_index)

        if initialize:
            if in_out_layers == False: # only initialize mask when it is not the input or output layer
                mask[layer_name] = np.ones(layer.shape[1:])
        
        if in_out_layers == True:
            layer = Multiply()([layer, tf.multiply(tf.ones_like(layer), tf.ones_like(layer))]) # Does nothing but retain the layer the way it was
        else:
            # apply mask to the layer to drop or retain nodes
            layer = Multiply()([layer, tf.multiply(tf.ones_like(layer), mask[layer_name])]) # tf.multiply converts to inputs same type - Tensors and ensures they are of same size by usig broadcasting
        
        activation_array.append(layer)
        layer_index += 1

        return mask, layer, activation_array, layer_index

    def add_filter_mask(self, mask, layer, layer_index, activation_array, initialize=True):
        """Initialize mask for layers in CNNs with certain mask.
        Args:
            mask: Dict - key is a layer, value is numpy array of containing 
                0s(nodes to be removed) or 1s(nodes to keep)
            layer: tf.keras layer
            layer_index: Int - unique number used in naming model layers
            activation_array: array of activations
            initialize: Boolean - If true initializes mask to all 1s, else uses the
                        mask corresponding to the current layer(i.e model) to drop 
                        nodes in the layer by doing element wise multiplication. 
        Returns:
            mask: Dict - key is a layer, value is numpy array of mask
            layer: updated tf.keras layer with mask
        """

        layer_name = 'layer_' + str(layer_index)

        if initialize:
            mask[layer_name] = np.ones(layer.shape[3:])
    
        # apply mask to the layer to drop or retain nodes
        layer = Multiply()([layer, tf.multiply(tf.ones_like(layer), mask[layer_name].reshape(1, 1, layer.shape[3]))])

        activation_array.append(layer)
        layer_index += 1

        return mask, layer, activation_array, layer_index

    def get_mask(self, mask, flattened_mask, post_activation_values, mask_type = 'min', percentile=0.2, drop_one=False, print_importance=False):
        """ Updates mask after each training cycle
        Args:
            mask: Dict - current mask
            flattened_mask - List of flattened masked indices per layer (1 for mask, 0 for no mask)
            post_activation_values - list of feature maps per layer
            mask_type: String - type of mask: min, max, random, min_layer, max_layer, random_layer
            percentile - percentage of channels/feature maps remaining to be masked
            print_importance - Boolean. Whether to print the importance scores per layer
            drop_one - Boolean. Whether to drop only one channel at a time
        Returns:
            mask: Dict - final masks after masking percentile proportion of remaining channels
        """

        node_importance = []
        layer_nodes = {}
        layer_names = list(mask.keys())
        
        # if only drop one, then percentile is 0
        if drop_one:
            percentile = 0
        
        for i in range(1, len(post_activation_values) - 1):
            prev_layer = tf.reduce_sum(tf.abs(post_activation_values[i - 1]), axis=0, keepdims=True) #sum across columns. Across examples.
            prev_layer = tf.reduce_sum(prev_layer, axis=1) #sum across rows

            next_layer = tf.reduce_sum(tf.abs(post_activation_values[i + 1]), axis=0, keepdims=True)  #sum across columns. Across examples.
            next_layer = tf.reduce_sum(next_layer, axis=1) #sum across rows

            #Now sum prev and next 
            concat_prev_and_next = tf.concat([prev_layer, next_layer], axis=0)
            sum_prev_and_next = tf.reduce_sum(concat_prev_and_next)

            # add current layer nodes across examples
            current_layer = tf.reduce_sum(tf.abs(post_activation_values[i]), axis=0) #sum across columns. Across examples

            layer_node_importance = tf.multiply(current_layer, sum_prev_and_next)
            layer_nodes[layer_names[i - 1]] = layer_node_importance
            node_importance = np.hstack([node_importance, layer_node_importance])
            if print_importance:
                print('Layer node importance: ', node_importance)

        # remove only those in maskindex
        flattened_mask = np.ravel(np.where(flattened_mask == 1))

        # find out the threshold node/filter value to remove
        if len(flattened_mask) > 0:
            # for max mask
            if mask_type == 'max':
                sorted_values = -np.sort(-node_importance[flattened_mask]) 
                index = int((percentile) * len(sorted_values))
                max_index = sorted_values[index]
            # for min or % mask
            else:
                sorted_values = np.sort(node_importance[flattened_mask])
                index = int(percentile * len(sorted_values))
                max_index = sorted_values[index]
        
        # Calculate the number of nodes to remove
        num_mask = 0

        for v in mask.values():
            num_mask += np.sum(v) 

        total_nodes = int((percentile) * num_mask)

        if drop_one:
            total_nodes = 1

        # remove at least one node
        if (total_nodes == 0):
            total_nodes = 1

        # identify the indices to drop for random mask
        if mask_type == 'random': # random is the only thing I don't understand in this work
            indices = np.random.permutation(flattened_mask)
            # take only the first total_nodes number of nodes
            indices = indices[:total_nodes]
            
            dropmaskindex = {}
            start_index = 0
            # assign nodes/filters to drop for each layer in dropmaskindex
            for k, v in mask.items():
                num_mask += np.sum(v)
                dropmaskindex[k] = indices[(indices >= start_index) & (indices < start_index + len(v))] - start_index
                start_index += len(v)

        for i, layer_node_importance in layer_nodes.items():
            #only if there is something to drop in current mask
            if(np.sum(mask[i])>0):
                if mask_type == 'max': 
                    indices = np.ravel(np.where(layer_node_importance >= max_index)) #Get indices of node values >= the threshold value
                    current_indices = np.ravel(np.where(mask[i].ravel())) #Get the mask indices
                    indices = [j for j in indices if j in current_indices]     
                # global random mask or layer random mask
                elif mask_type == 'random_layer':
                    indices = np.ravel(np.where(mask[i].ravel()))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                elif mask_type == 'random':
                    indices = dropmaskindex[i]
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                # layer-wise max mask
                elif mask_type == 'max_layer':
                    sorted_values = -np.sort(-layer_node_importance[mask[i]==1])
                    index = int((percentile) * len(sorted_values))
                    maxindex = sorted_values[index]
                    indices = np.ravel(np.where(layer_node_importance >= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
                # layer-wise min mask
                elif mask_type == 'min_layer':
                    sorted_values = np.sort(layer_node_importance[mask[i]==1])
                    index = int((percentile) * len(sorted_values))
                    max_index = sorted_values[index]
                    indices = np.ravel(np.where(layer_node_importance <= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
                # if this is min mask or % based mask
                else:
                    indices = np.ravel(np.where(layer_node_importance <= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
            else:
                #default
                indices = np.ravel(np.where(mask[i]==1))

            # shuffle the indices only if we are not dropping one node/filter
            if (drop_one == False):
                indices = np.random.permutation(indices)
            
            # Get the current mask ready for update by making a copy
            new_mask = mask[i].ravel()

            # for layer masks, total nodes dropped is by percentile of the layer of each mask
            if(mask_type == 'random_layer') or mask_type == 'min_layer' or mask_type == 'max_layer':
                initial_percent = np.sum(mask[i]) * 1.0 / len(mask[i].ravel())
                total_nodes = int(initial_percent * (percentile) * len(mask[i].ravel()))

                # remove at least 1 node
                if (total_nodes == 0):
                    total_nodes = 1
            
            if(len(indices) > 0):
                # remove at most total_nodes number of nodes
                if(len(indices) > total_nodes):
                    indices = indices[:total_nodes]

                # remove nodes
                new_mask[indices] = 0

                # update total_nodes to be removed
                total_nodes = total_nodes - len(indices)
            
            # reshape to fit new mask
            mask[i] = new_mask.reshape(mask[i].shape)

        return mask

    def get_mask_cnn(self, mask, flattened_mask, activation_model, x_train, y_train, loss_function, mask_type = 'min', percentile=0.2, drop_one=False, print_importance=False):
        """ Updates mask after each training cycle
        Args:
            mask: Dict - current mask
            flattened_mask - List of flattened masked indices per layer (1 for mask, 0 for no mask)
            activation_model - tf.keras.Model of post activations
            post_activation_values - list of feature maps per layer
            mask_type: String - type of mask: min, max, random, min_layer, max_layer, random_layer
            percentile - percentage of channels/feature maps remaining to be masked
            print_importance - Boolean. Whether to print the importance scores per layer
            drop_one - Boolean. Whether to drop only one channel at a time
        Returns:
            mask: Dict - final masks after masking percentile proportion of remaining channels
        """
        
        layer_names = list(mask.keys())
        layer_scores = {}
        importance = []

        # if only drop one, then percentile is 0
        if drop_one:
            percentile = 0
            
        
        #batch dataset for faster computation 
        X_train = tf.data.Dataset.from_tensor_slices(x_train)
        Y_train = tf.data.Dataset.from_tensor_slices(y_train)
        train_dataset = tf.data.Dataset.zip((X_train, Y_train))
        batch_size = 256
        batches = train_dataset.batch(batch_size)

        for step, (x_train, y_train) in enumerate(batches, 1):
            with tf.GradientTape(persistent=True) as tape:
                with tf.device('/cpu:0'):
                    activations = activation_model(x_train)
                    loss = loss_function(y_train, activations[-1])
                    #print(f'loss (y, predicted(output layer)): {loss}\n')

            for i in range(len(activations) - 1):
                current_layer = tape.gradient(loss, activations[i])
                #print(f'layer dl/d_{layer_names[i]}: {current_layer.shape}')

                # sum across examples (columns)
                current_layer_sensitivity =  tf.reduce_sum(tf.abs(current_layer), axis=0).numpy()
                #print(f'current layer sensitivity: {current_layer_sensitivity.shape}')
                if layer_names[i] in layer_scores:
                    layer_scores[layer_names[i]] += current_layer_sensitivity
                else:
                    layer_scores[layer_names[i]] = current_layer_sensitivity

        number_examples = batch_size * step
        for layer, layer_sums in layer_scores.items():
            #mean of each constituent gradient in a feature map across all examples 
            node_mean_across_examples = layer_sums / number_examples 
            layer_scores[layer] = tf.reduce_mean(node_mean_across_examples, axis=[0, 1])
            importance = np.hstack([importance, layer_scores[layer]])
        
        if print_importance:
            print(f'importance: {importance}')

        # remove only those in maskindex
        flattened_mask = np.ravel(np.where(flattened_mask == 1))

        # find out the threshold feature map to remove
        if len(flattened_mask) > 0:
            # for max mask
            if mask_type == 'max':
                sorted_values = -np.sort(-importance[flattened_mask]) 
                index = int((percentile) * len(sorted_values))
                max_index = sorted_values[index]
            # for min or % mask
            else:
                sorted_values = np.sort(importance[flattened_mask])
                index = int(percentile * len(sorted_values))
                max_index = sorted_values[index]
        
        # Calculate the number of nodes to remove
        num_mask = 0

        for v in mask.values():
            num_mask += np.sum(v) 

        total_nodes = int((percentile) * num_mask)

        if drop_one:
            total_nodes = 1

        # remove at least one node
        if (total_nodes == 0):
            total_nodes = 1

        # identify the indices to drop for random mask
        if mask_type == 'random': # random is the only thing I don't understand in this work
            indices = np.random.permutation(flattened_mask)
            # take only the first total_nodes number of nodes
            indices = indices[:total_nodes]
            
            dropmaskindex = {}
            start_index = 0
            # assign nodes/filters to drop for each layer in dropmaskindex
            for k, v in mask.items():
                num_mask += np.sum(v)
                dropmaskindex[k] = indices[(indices >= start_index) & (indices < start_index + len(v))] - start_index
                start_index += len(v)

        for i, layer_importance in layer_scores.items():
            #only if there is something to drop in current mask
            if(np.sum(mask[i])>0):
                if mask_type == 'max': 
                    indices = np.ravel(np.where(layer_importance >= max_index)) #Get indices of node values >= the threshold value
                    current_indices = np.ravel(np.where(mask[i].ravel())) #Get the mask indices
                    indices = [j for j in indices if j in current_indices]     
                # global random mask or layer random mask
                elif mask_type == 'random_layer':
                    indices = np.ravel(np.where(mask[i].ravel()))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                elif mask_type == 'random':
                    indices = dropmaskindex[i]
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                # layer-wise max mask
                elif mask_type == 'max_layer':
                    sorted_values = -np.sort(-layer_importance[mask[i]==1])
                    index = int((percentile) * len(sorted_values))
                    max_index = sorted_values[index]
                    indices = np.ravel(np.where(layer_importance >= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
                # layer-wise min mask
                elif mask_type == 'min_layer':
                    sorted_values = np.sort(layer_importance[mask[i]==1])
                    index = int((percentile) * len(sorted_values))
                    max_index = sorted_values[index]
                    indices = np.ravel(np.where(layer_importance <= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
                # if this is min mask or % based mask
                else:
                    indices = np.ravel(np.where(layer_importance <= max_index))
                    current_indices = np.ravel(np.where(mask[i].ravel()))
                    indices = [j for j in indices if j in current_indices]
            else:
                #default
                indices = np.ravel(np.where(mask[i]==1))

            # shuffle the indices only if we are not dropping one filter
            if (drop_one == False):
                indices = np.random.permutation(indices)
            
            # Get the current mask ready for update by making a copy
            new_mask = mask[i].ravel()

            # for layer masks, total nodes dropped is by percentile of the layer of each mask
            if(mask_type == 'random_layer') or mask_type == 'min_layer' or mask_type == 'max_layer':
                initial_percent = np.sum(mask[i]) * 1.0 / len(mask[i].ravel())
                total_nodes = int(initial_percent * (percentile) * len(mask[i].ravel()))

                # remove at least 1 node
                if (total_nodes == 0):
                    total_nodes = 1
            
            if(len(indices) > 0):
                # remove at most total_nodes number of nodes
                if(len(indices) > total_nodes):
                    indices = indices[:total_nodes]

                # remove nodes
                new_mask[indices] = 0

                # update total_nodes to be removed
                total_nodes = total_nodes - len(indices)
            
            # reshape to fit new mask
            mask[i] = new_mask.reshape(mask[i].shape)

        return mask

    def generate_model(self, experiment_name, opt = SGD(lr = 0.1), loss = 'sparse_categorical_crossentropy',
                        metrics = ['accuracy'], callbacks = [EarlyStopping(monitor='val_loss', mode = 'min', verbose=0, patience=5)], 
                        num_epochs = 100, percentile = 0.2, verbose = 0, num_trials = 15, print_value = True, 
                        print_mask = False, print_importance = False, cnn=True):
        """ Function to evaluate the model against various metrics
        Inputs:
        opt - Keras optimizer
        loss - Keras loss
        metrics - Keras metrics
        callbacks - Keras callbacks
        num_epochs - Number of epochs for each training cycle
        percentile - Percentile to drop the nodes
        verbose - Keras verbose option
        num_trials - Number of different experiments to run, each with different random seed
        print_value  - Boolean. Whether to print the accuracies and losses for each pruning percentage
        print_mask - Boolean. Whether to print the mask values
        print_activation - Boolean. Whether to print the node/filter's activation values
        
        Output:
        caches - Cache containing accuracies, losses, early stopping iteration and oracle comparison
        """

        # Initialize variables
        percent_removed ={}
        train_accuracies = {}
        valid_accuracies = {}
        test_accuracies = {}
        train_losses = {}
        valid_losses ={}
        test_losses = {}
        early_stopping = {}
        oracle_comparison = {}
        masktypes = ['min', 'max', 'random', 'min_layer', 'max_layer', 'random_layer']
        
        for masktype in masktypes:
            percent_removed[masktype] = []
            train_accuracies[masktype] = []
            valid_accuracies[masktype] = []
            test_accuracies[masktype] = []
            train_losses[masktype] = []
            valid_losses[masktype] = []
            test_losses[masktype] = []
            early_stopping[masktype] = []
            
        # do for num_trials number of random seeds
        for random_seed in range(num_trials):
            print('>>>     Random seed number:', random_seed)

            np.random.seed(random_seed)
            tf.random.set_seed(random_seed)    
            
            # Initialize the model
            mask, model, activation_model = self.initialize_model()
            # model.summary()
            model.compile(optimizer = opt, loss = loss, metrics = metrics)
            # save original weights
            weights_initial = model.get_weights()
        
        # do for all mask types
            for masktype in masktypes:
                print('\n>>> Currently doing', masktype, 'mask <<<')
            
                mask=self.reset_mask(mask)
                percent = self.percent_mask(mask)
                
                while percent > 0.1:
                    # Initialize the model with the new mask
                    tf.keras.backend.clear_session()
                    np.random.seed(random_seed)
                    tf.random.set_seed(random_seed)  
                    
                    _, model, activation_model = self.initialize_model(mask)

                    #Initialize to original weights
                    model.set_weights(weights_initial)
                    model.compile(optimizer = opt, loss = loss, metrics = metrics)  

                    history = model.fit(self.x_train, self.y_train, epochs = num_epochs, validation_data = (self.x_val, self.y_val), callbacks = callbacks, shuffle = False, verbose = verbose)
                    results = model.evaluate(self.x_test, self.y_test, verbose = 0)

                    percent = self.percent_mask(mask)
                    train_accuracy = history.history['accuracy'][-1]
                    valid_accuracy = history.history['val_accuracy'][-1]
                    test_accuracy = results[1]
                    train_loss = history.history['loss'][-1]
                    valid_loss = history.history['val_loss'][-1]
                    test_loss = results[0]
                    early = len(history.history['accuracy'])

                    # Append the values for accuracy and loss
                    percent_removed[masktype].append(percent)
                    train_accuracies[masktype].append(train_accuracy)
                    valid_accuracies[masktype].append(valid_accuracy)
                    test_accuracies[masktype].append(test_accuracy)
                    train_losses[masktype].append(train_loss)
                    valid_losses[masktype].append(valid_loss)
                    test_losses[masktype].append(test_loss)
                    early_stopping[masktype].append(early)

                    if print_value:
                        print('Percentage remaining', percent, end = ' ')
                        print('Layer nodes:', [np.sum(mask[i]) for i in mask.keys()], end = ' ')
                        if print_mask:
                            print('Mask:', mask)
                        print('Train Acc:', train_accuracy, end = ' ')
                        print('Val Acc:', valid_accuracy, end = ' ')
                        print('Test Acc:', test_accuracy)
                        print('Train Loss:', train_loss, end = ' ')
                        print('Val loss:', valid_loss, end = ' ')
                        print('Test Loss:', test_loss)
                        print('Early stopping iteration:', early)

                    # Remove nodes for next iteration based on metric
                    post_activation_values = activation_model.predict(self.x_train)
                    loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
                    flattened_mask = self.flatten_mask(mask)
                    if cnn:
                        mask = self.get_mask_cnn(mask, flattened_mask, activation_model, self.x_train, self.y_train, loss_function, mask_type=masktype, percentile=percentile, print_importance=print_importance)
                    else:
                        mask = self.get_mask(mask, flattened_mask, post_activation_values, mask_type=masktype, percentile=percentile, print_importance=print_importance)
                    

            cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
            self.print_graph(cache, 'evaluate'+str(random_seed), experiment_name, num_trials = random_seed+1)

        cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
        return cache

    def compare_random(self, experiment_name, opt = SGD(lr = 0.1),
                        loss = 'sparse_categorical_crossentropy',
                        callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5)], 
                        metrics = ['accuracy'], num_epochs = 100, percentile = 0.2, verbose = 0, num_trials = 15, 
                        print_value = True, print_mask = False, print_importance = False, cnn=True):
        """ Function to compare metrics with randominit 
        Inputs:
        opt - Keras optimizer
        loss - Keras loss
        callbacks - Keras callbacks
        metrics - Keras metrics
        num_epochs - Number of epochs for each training cycle
        percentile - Percentile to drop the nodes
        verbose - Keras verbose option
        num_trials - Number of different experiments to run, each with different random seed
        print_value  - Boolean. Whether to print the accuracies and losses for each pruning percentage
        print_mask - Boolean. Whether to print the mask values
        print_importance - Boolean. Whether to print the node/filter's activation values
        
        Output:
        caches - Cache containing accuracies, losses, early stopping iteration and oracle comparison
        """

        # Initialize variables
        percent_removed ={}
        train_accuracies = {}
        valid_accuracies = {}
        test_accuracies = {}
        train_losses = {}
        valid_losses ={}
        test_losses = {}
        early_stopping = {}
        oracle_comparison = {}
        masktypes = ['min', 'randominit']
        
        for masktype in masktypes:
            percent_removed[masktype] = []
            train_accuracies[masktype] = []
            valid_accuracies[masktype] = []
            test_accuracies[masktype] = []
            train_losses[masktype] = []
            valid_losses[masktype] = []
            test_losses[masktype] = []
            early_stopping[masktype] = []
            
        # do for num_trials number of random seeds
        for random_seed in range(num_trials):
            print('>>>     Random seed number:', random_seed)

            np.random.seed(random_seed)
            tf.random.set_seed(random_seed)   
            
            # Initialize the model
            mask, model, activation_model = self.initialize_model()
            # model.summary()
            model.compile(optimizer = opt, loss = loss, metrics = metrics)
            
            # save original weights
            weights_initial = model.get_weights()

            mask = self.reset_mask(mask)
            percent = self.percent_mask(mask)
            
            while percent > 0.1:
                print('\n>>> Currently doing min mask <<<')
                tf.keras.backend.clear_session()
                np.random.seed(random_seed)
                tf.random.set_seed(random_seed)  
                
                # Initialize the model with the new mask
                _, model, activation_model = self.initialize_model(mask)

                #Initialize to original weights
                model.set_weights(weights_initial)
                model.compile(optimizer = opt, loss = loss, metrics = metrics)

                history = model.fit(self.x_train, self.y_train, epochs = num_epochs, validation_data = (self.x_val, self.y_val), shuffle = False, callbacks = callbacks, verbose = verbose)
                results = model.evaluate(self.x_test, self.y_test, verbose = 0)

                percent = self.percent_mask(mask)
                train_accuracy = history.history['accuracy'][-1]
                valid_accuracy = history.history['val_accuracy'][-1]
                test_accuracy = results[1]
                train_loss = history.history['loss'][-1]
                valid_loss = history.history['val_loss'][-1]
                test_loss = results[0]
                early = len(history.history['accuracy'])

                # Append the values for accuracy and loss
                percent_removed['min'].append(percent)
                train_accuracies['min'].append(train_accuracy)
                valid_accuracies['min'].append(valid_accuracy)
                test_accuracies['min'].append(test_accuracy)
                train_losses['min'].append(train_loss)
                valid_losses['min'].append(valid_loss)
                test_losses['min'].append(test_loss)
                early_stopping['min'].append(early)

                if print_value:
                    print('Percentage remaining', percent, end = ' ')
                    print('Layer nodes:', [np.sum(mask[i]) for i in mask.keys()], end = ' ')
                    if print_mask:
                        print('Mask:', mask)
                    print('Train Acc:', train_accuracy, end = ' ')
                    print('Val Acc:', valid_accuracy, end = ' ')
                    print('Test Acc:', test_accuracy)
                    print('Train Loss:', train_loss, end = ' ')
                    print('Val loss:', valid_loss, end = ' ')
                    print('Test Loss:', test_loss)
                    print('Early stopping iteration:', early)
                    
                # Remove nodes for next iteration based on metric
                post_activation_values = activation_model.predict(self.x_train)
                flattened_mask = self.flatten_mask(mask)

                # random init mask
                print('\n>>> Currently doing randominit mask <<<')
                # Initialize the model with the new mask with random seed
                tf.keras.backend.clear_session()
                np.random.seed(random_seed+20)
                tf.random.set_seed(random_seed+20)    
                _, model, activation_model = self.initialize_model(mask)

                model.compile(optimizer = opt, loss = loss, metrics = metrics)

                history = model.fit(self.x_train, self.y_train, epochs = num_epochs, validation_data = (self.x_val, self.y_val), shuffle = False, callbacks = callbacks, verbose = verbose)
                results = model.evaluate(self.x_test, self.y_test, verbose = 0)

                percent = self.percent_mask(mask)
                train_accuracy = history.history['accuracy'][-1]
                valid_accuracy = history.history['val_accuracy'][-1]
                test_accuracy = results[1]
                train_loss = history.history['loss'][-1]
                valid_loss = history.history['val_loss'][-1]
                test_loss = results[0]
                early = len(history.history['accuracy'])

                # Append the values for accuracy and loss
                percent_removed['randominit'].append(percent)
                train_accuracies['randominit'].append(train_accuracy)
                valid_accuracies['randominit'].append(valid_accuracy)
                test_accuracies['randominit'].append(test_accuracy)
                train_losses['randominit'].append(train_loss)
                valid_losses['randominit'].append(valid_loss)
                test_losses['randominit'].append(test_loss)
                early_stopping['randominit'].append(early)

                if print_value:
                    print('Percentage remaining', percent, end = ' ')
                    print('Layer nodes:', [np.sum(mask[i]) for i in mask.keys()], end = ' ')
                    if print_mask:
                        print('Mask:', mask)
                    print('Train Acc:', train_accuracy, end = ' ')
                    print('Val Acc:', valid_accuracy, end = ' ')
                    print('Test Acc:', test_accuracy)
                    print('Train Loss:', train_loss, end = ' ')
                    print('Val loss:', valid_loss, end = ' ')
                    print('Test Loss:', test_loss)
                    print('Early stopping iteration:', early)
                
                loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
                # get new mask
                if cnn:
                    mask = self.get_mask_cnn(mask, flattened_mask, activation_model, self.x_train, self.y_train, loss_function, mask_type='min', percentile=percentile, print_importance=print_importance)
                else:
                    mask = self.get_mask(mask, flattened_mask, post_activation_values, mask_type='min', percentile=percentile, print_importance=print_importance)
    

            cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
            self.print_graph(cache, 'randominit'+str(random_seed), experiment_name, num_trials = random_seed+1)

        cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
        return cache 

    def oracle_model(self, experiment_name, opt = SGD(lr = 0.1), loss = 'sparse_categorical_crossentropy',
                    metrics = ['accuracy'],
                    callbacks = [EarlyStopping(monitor='val_loss', mode = 'min', verbose=0, patience=5)], 
                    num_epochs = 100, percentile = 0.2, verbose = 0, num_trials = 15, print_value = True, 
                    print_mask = False, print_importance = False, cnn=True):
        
        """ Function to evaluate the model against the oracle
        Inputs:
        opt - Keras optimizer
        loss - Keras loss
        metrics - Keras metrics
        callbacks - Keras callbacks
        num_epochs - Number of epochs for each training cycle
        percentile - Percentile to drop the nodes
        verbose - Keras verbose option
        num_trials - Number of different experiments to run, each with different random seed
        print_value  - Boolean. Whether to print the accuracies and losses for each pruning percentage
        print_mask - Boolean. Whether to print the mask values
        print_importance - Boolean. Whether to print the node/filter's activation values
        
        Output:
        caches - Cache containing accuracies, losses, early stopping iteration and oracle comparison
        """

        # Initialize variables
        percent_removed ={}
        train_accuracies = {}
        valid_accuracies = {}
        test_accuracies = {}
        train_losses = {}
        valid_losses ={}
        test_losses = {}
        early_stopping = {}
        oracle_comparison = {}
        oracle_masks = []
        masktypes = ['oracle', 'min', 'max', 'random_layer']
        
        for masktype in masktypes:
            percent_removed[masktype] = []
            train_accuracies[masktype] = []
            valid_accuracies[masktype] = []
            test_accuracies[masktype] = []
            train_losses[masktype] = []
            valid_losses[masktype] = []
            test_losses[masktype] = []
            early_stopping[masktype] = []
            oracle_comparison[masktype] = []
            
        # do for num_trials number of random seeds
        for random_seed in range(num_trials):
            print('>>>     Random seed number:', random_seed)

            np.random.seed(random_seed)
            tf.random.set_seed(random_seed)    
            
            # Initialize the model
            mask, model, activation_model = self.initialize_model()
            # model.summary()
            model.compile(optimizer = opt, loss = loss, metrics = metrics)
            # save original weights
            weights_initial = model.get_weights()
            
            # reset oracle masks
            oracle_masks = []
                    
            # do for all mask types
            for masktype in masktypes:
                print('\n>>> Currently doing', masktype, 'mask <<<')
            
                mask=self.reset_mask(mask)
                percent = self.percent_mask(mask)
                iterationnum = 0
                
                while percent > 0.1:
                    # Initialize the model with the new mask
                    tf.keras.backend.clear_session()
                    np.random.seed(random_seed)
                    tf.random.set_seed(random_seed)  
                    
                    _, model, activation_model = self.initialize_model(mask)
                    
                    #Initialize to original weights
                    model.set_weights(weights_initial)
                    model.compile(optimizer = opt, loss = loss, metrics = metrics)  

                    history = model.fit(self.x_train, self.y_train, epochs = num_epochs, validation_data = (self.x_val, self.y_val), callbacks = callbacks, shuffle = False, verbose = verbose)
                    results = model.evaluate(self.x_test, self.y_test, verbose = 0)
                    
                    weights_after_training = model.get_weights()

                    percent = self.percent_mask(mask)
                    train_accuracy = history.history['accuracy'][-1]
                    valid_accuracy = history.history['val_accuracy'][-1]
                    test_accuracy = results[1]
                    train_loss = history.history['loss'][-1]
                    valid_loss = history.history['val_loss'][-1]
                    test_loss = results[0]
                    early = len(history.history['accuracy'])

                    # Append the values for accuracy and loss
                    percent_removed[masktype].append(percent)
                    train_accuracies[masktype].append(train_accuracy)
                    valid_accuracies[masktype].append(valid_accuracy)
                    test_accuracies[masktype].append(test_accuracy)
                    train_losses[masktype].append(train_loss)
                    valid_losses[masktype].append(valid_loss)
                    test_losses[masktype].append(test_loss)
                    early_stopping[masktype].append(early)
                    
                    # Compare mask with oracle if this is iteration 1 and above (when there is a mask to compare)
                    if iterationnum == 0:
                        compare = 1
                    else:
                        compare = self.compare_mask(mask, oracle_masks[iterationnum-1])
                    oracle_comparison[masktype].append(compare)

                    if print_value:
                        print('Percentage remaining', percent, end = ' ')
                        print('Layer nodes:', [np.sum(mask[i]) for i in mask.keys()], end = ' ')
                        if print_mask:
                            print('Mask:', mask)
                        print('Train Acc:', train_accuracy, end = ' ')
                        print('Val Acc:', valid_accuracy, end = ' ')
                        print('Test Acc:', test_accuracy)
                        print('Train Loss:', train_loss, end = ' ')
                        print('Val loss:', valid_loss, end = ' ')
                        print('Test Loss:', test_loss)
                        print('Early stopping iteration:', early)
                        print('Similaritiy with oracle:', compare)

                    # Remove nodes for next iteration for oracle by doing brute force comparison
                    if masktype == 'oracle':
                        bestloss = 1000000
                        bestmask = {}
                        
                        # Remove one existing node and compare accuracy
                        for v in mask.values():
                            for index in range(len(v)):
                                # remove mask index only if it is non-zero
                                if v[index] != 0:
                                    
                                    # take out mask index
                                    v[index] = 0
                                    
                                    # do the mask evaluation
                                    # Initialize the model with the new mask
                                    _, model, activation_model = self.initialize_model(mask)

                                    #Initialize the weights to after training weights
                                    model.set_weights(weights_after_training)
                                    model.compile(optimizer = opt, loss = loss, metrics = metrics)
                                    # evaluate model using training data only
                                    results = model.evaluate(self.x_train, self.y_train, verbose = 0)
                                    
                                    # update accuracy and mask for best mask
                                    if(results[0] < bestloss):
                                        bestloss = results[0]
                                        bestmask = copy.deepcopy(mask)
                                            
                                    # put back mask index
                                    v[index] = 1
                            
                        # store bestmask
                        mask = copy.deepcopy(bestmask)
                        oracle_masks.append(mask)
                        
                        if print_value:
                            print('Final mask selected:', mask)
                        oracle_comparison[masktype].append(self.compare_mask(mask, oracle_masks[iterationnum]))

                    else:
                        # Remove nodes for next iteration based on metric
                        loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
                        post_activation_values = activation_model.predict(self.x_train)
                        flattened_mask = self.flatten_mask(mask)
                        if cnn:
                            mask = self.get_mask_cnn(mask, flattened_mask, activation_model, self.x_train, self.y_train, loss_function, percentile=percentile, drop_one=True, print_importance=print_importance)
                        else:
                            mask = self.get_mask(mask, flattened_mask, post_activation_values, percentile=percentile, drop_one=True, print_importance=print_importance)
            
                        oracle_comparison[masktype].append(self.compare_mask(mask, oracle_masks[iterationnum]))
                    
                    # Increment iteration number
                    iterationnum = iterationnum + 1
                    
            cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
            self.print_graph(cache, 'oracle'+str(random_seed), experiment_name, num_trials = random_seed+1)

        cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
        return cache

    def compare_percent_mask(self, experiment_name, opt = SGD(lr = 0.1), loss = 'sparse_categorical_crossentropy',
                            metrics = ['accuracy'],
                            callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=5)], 
                            num_epochs = 100, percentile = 0.2, verbose = 0, num_trials = 15, print_value = True, 
                            print_mask = False, print_importance = False, cnn=True):
        """ Function to compare the effect of pruning proportions
        Inputs:
        opt - Keras optimizer
        loss - Keras loss
        metrics - Keras metrics
        callbacks - Keras callbacks
        num_epochs - Number of epochs for each training cycle
        percentile - Percentile to drop the nodes
        verbose - Keras verbose option
        num_trials - Number of different experiments to run, each with different random seed
        print_value  - Boolean. Whether to print the accuracies and losses for each pruning percentage
        print_mask - Boolean. Whether to print the mask values
        print_importance - Boolean. Whether to print the node/filter's activation values
        
        Output:
        caches - Cache containing accuracies, losses, early stopping iteration and oracle comparison
        """

        # Initialize variables
        percent_removed ={}
        train_accuracies = {}
        valid_accuracies = {}
        test_accuracies = {}
        train_losses = {}
        valid_losses ={}
        test_losses = {}
        early_stopping = {}
        oracle_comparison = {}
        masktypes = ['0.2', '0.3', '0.4', '0.5', '0.9']
        
        for masktype in masktypes:
            percent_removed[masktype] = []
            train_accuracies[masktype] = []
            valid_accuracies[masktype] = []
            test_accuracies[masktype] = []
            train_losses[masktype] = []
            valid_losses[masktype] = []
            test_losses[masktype] = []
            early_stopping[masktype] = []
            
        # do for num_trials number of random seeds
        for random_seed in range(num_trials):
            print('>>>     Random seed number:', random_seed)

            np.random.seed(random_seed)
            tf.random.set_seed(random_seed)    
            
            # Initialize the model
            mask, model, activation_model = self.initialize_model()
            # model.summary()
            model.compile(optimizer = opt, loss = loss, metrics = metrics)
            # save original weights
            weights_initial = model.get_weights()
        
        # do for all mask types
            for masktype in masktypes:
                print('\n>>> Currently doing', masktype, 'mask <<<')
            
                mask=self.reset_mask(mask)
                percent = self.percent_mask(mask)
                
                while percent > 0.1:
                    # Initialize the model with the new mask
                    tf.keras.backend.clear_session()
                    np.random.seed(random_seed)
                    tf.random.set_seed(random_seed)  
                    
                    _, model, activation_model = self.initialize_model(mask)

                    #Initialize to original weights
                    model.set_weights(weights_initial)
                    model.compile(optimizer = opt, loss = loss, metrics = metrics)  

                    history = model.fit(self.x_train, self.y_train, epochs = num_epochs, validation_data = (self.x_val, self.y_val), callbacks = callbacks, shuffle = False, verbose = verbose)
                    results = model.evaluate(self.x_test, self.y_test, verbose = 0)

                    percent = self.percent_mask(mask)
                    train_accuracy = history.history['accuracy'][-1]
                    valid_accuracy = history.history['val_accuracy'][-1]
                    test_accuracy = results[1]
                    train_loss = history.history['loss'][-1]
                    valid_loss = history.history['val_loss'][-1]
                    test_loss = results[0]
                    early = len(history.history['accuracy'])

                    # Append the values for accuracy and loss
                    percent_removed[masktype].append(percent)
                    train_accuracies[masktype].append(train_accuracy)
                    valid_accuracies[masktype].append(valid_accuracy)
                    test_accuracies[masktype].append(test_accuracy)
                    train_losses[masktype].append(train_loss)
                    valid_losses[masktype].append(valid_loss)
                    test_losses[masktype].append(test_loss)
                    early_stopping[masktype].append(early)

                    if print_value:
                        print('Percentage remaining', percent, end = ' ')
                        print('Layer nodes:', [np.sum(mask[i]) for i in mask.keys()], end = ' ')
                        if print_mask:
                            print('Mask:', mask)
                        print('Train Acc:', train_accuracy, end = ' ')
                        print('Val Acc:', valid_accuracy, end = ' ')
                        print('Test Acc:', test_accuracy)
                        print('Train Loss:', train_loss, end = ' ')
                        print('Val loss:', valid_loss, end = ' ')
                        print('Test Loss:', test_loss)
                        print('Early stopping iteration:', early)
                        
                    # get the percentile from masktype
                    if masktype == '0.1': 
                        percentile = 0.1
                    elif masktype == '0.2': 
                        percentile = 0.2
                    elif masktype == '0.3': 
                        percentile = 0.3
                    elif masktype == '0.4': 
                        percentile = 0.4
                    elif masktype == '0.5': 
                        percentile = 0.5
                    elif masktype == '0.9': 
                        percentile = 0.9

                    # Remove nodes for next iteration based on metric
                    loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
                    post_activation_values = activation_model.predict(self.x_train)
                    flattened_mask = self.flatten_mask(mask)
                    if cnn:
                        mask = self.get_mask_cnn(mask, flattened_mask, activation_model, self.x_train, self.y_train, loss_function, mask_type='min', percentile=percentile, print_importance=print_importance)
                    else:
                        mask = self.get_mask(mask, flattened_mask, post_activation_values, mask_type='min', percentile=percentile, print_importance=print_importance)

                    
            cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
            self.print_graph(cache, 'comparepercentmask'+str(random_seed), experiment_name, num_trials = random_seed+1)
                    
        cache = (percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison)
        return cache

    def print_graph(self, cache, name, experiment_name, num_trials = 15, oracle = False):
        """Function to print graph
        Input: 
        cache - Cache containing accuracies, losses, early stopping iteration and oracle comparison
        name - name of model which is run
        num_trials - number of experiments conducted
        oracle - Boolean. Whether the oracle_comparison graph is required
        
        Outputs:
        Graphs for training, validation, test accuracy, early stopping iteration, oracle comparison (optional)"""
        
        # unpack caches
        percent_removed, train_accuracies, valid_accuracies, test_accuracies, train_losses, valid_losses, test_losses, early_stopping, oracle_comparison = cache
        # sort masktypes by alphabetical order
        masktypes = sorted(percent_removed.keys())
        
        # Save result of each experiment in a folder
        if not os.path.exists(experiment_name):
            os.mkdir(experiment_name)

        # Set colors
        colors = {}
        colors['min'] = 'b'
        colors['max'] = 'r'
        colors['random'] ='g'
        colors['random_layer'] = 'y'
        colors['min_layer'] = 'c'
        colors['max_layer'] = 'm'
        colors['randominit'] = 'y'
        colors['oracle'] = 'm'
        
        # colors for percentage mask
        colors['0.1'] = 'b'
        colors['0.2'] = 'r'
        colors['0.3'] = 'g'
        colors['0.4'] = 'y'
        colors['0.5'] = 'c'
        colors['0.9'] = 'm'
        
        # format for various masks
        fmt = {}
        fmt['min'] = '-'
        fmt['max'] = '-'
        fmt['random'] ='--'
        fmt['random_layer'] = '--'
        fmt['min_layer'] = '-'
        fmt['max_layer'] = '-'
        fmt['randominit'] = '--'
        fmt['minfast'] = '-'
        fmt['oracle'] = '-'
        
        fmt['0.1'] = '-'
        fmt['0.2'] = '-'
        fmt['0.3'] = '-'
        fmt['0.4'] = '-'
        fmt['0.5'] = '-'
        fmt['0.9'] = '-'
        
        #import matplotlib.font_manager
        #from matplotlib import rc, rcParams
        rc('font', family = 'STIXGeneral')
        rc('xtick', labelsize=10) 
        rcParams.update({'figure.autolayout': True})
        rcParams.update({'font.size': 14})
        
        # Plot figures for training accuracy
        plt.figure()
        plt.gca().invert_xaxis()
        plt.xscale('log')
        plt.xticks([0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0], ['0.1','0.2','0.3','0.4','0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], rotation = 45)
        for masktype in masktypes:     
            length = len(percent_removed[masktype])//num_trials
            mean = []
            std = []
            for i in range(length):
                mean.append(np.mean(train_accuracies[masktype][i::length]))
                std.append(1.96*np.std(train_accuracies[masktype][i::length])/np.sqrt(num_trials))
            plt.errorbar(percent_removed[masktype][:length], mean, yerr = std, fmt = fmt[masktype], capsize = 2, alpha = 0.5, color=colors[masktype], label = masktype)        
        plt.ylabel('Training accuracy')
        plt.xlabel('Proportion of nodes/feature maps remaining')
        plt.legend(loc = 'lower left')
        plt.savefig(f'{experiment_name}/training_accuracy_{name}.png')

        # Plot figures for validation accuracy
        plt.figure()
        plt.gca().invert_xaxis()
        plt.xscale('log')
        plt.xticks([0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0], ['0.1','0.2','0.3','0.4','0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], rotation = 45)
        for masktype in masktypes:
            length = len(percent_removed[masktype])//num_trials
            mean = []
            std = []
            for i in range(length):
                mean.append(np.mean(valid_accuracies[masktype][i::length]))
                std.append(1.96*np.std(valid_accuracies[masktype][i::length])/np.sqrt(num_trials))
            plt.errorbar(percent_removed[masktype][:length], mean, yerr = std, fmt = fmt[masktype], capsize = 2, alpha = 0.5, color=colors[masktype], label = masktype)
        plt.ylabel('Validation accuracy')
        plt.xlabel('Proportion of nodes/feature maps remaining')
        plt.legend(loc = 'lower left')
        plt.savefig(f'{experiment_name}/validation_accuracy_{name}.png')

        # Plot figures for test accuracy
        plt.figure()
        plt.gca().invert_xaxis()
        plt.xscale('log')
        plt.xticks([0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0], ['0.1','0.2','0.3','0.4','0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], rotation = 45)
        for masktype in masktypes:
            length = len(percent_removed[masktype])//num_trials
            mean = []
            std = []
            for i in range(length):
                mean.append(np.mean(test_accuracies[masktype][i::length]))
                std.append(1.96*np.std(test_accuracies[masktype][i::length])/np.sqrt(num_trials))
            plt.errorbar(percent_removed[masktype][:length], mean, yerr = std, fmt = fmt[masktype], capsize = 2, alpha = 0.5, color=colors[masktype], label = masktype)
        plt.ylabel('Test accuracy')
        plt.xlabel('Proportion of nodes/feature maps remaining')
        plt.legend(loc = 'lower left')
        plt.savefig(f'{experiment_name}/test_accuracy_{name}.png')

        if oracle:
            # Plot figures for oracle comparison
            plt.figure()
            plt.gca().invert_xaxis()
            plt.xscale('log')
            plt.xticks([0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0], ['0.1','0.2','0.3','0.4','0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], rotation = 45)
            for masktype in masktypes:
                length = len(percent_removed[masktype])//num_trials
                mean = []
                std = []
                for i in range(length):
                    mean.append(np.mean(oracle_comparison[masktype][i::length]))
                    std.append(1.96*np.std(oracle_comparison[masktype][i::length])/np.sqrt(num_trials))
                plt.errorbar(percent_removed[masktype][:length], mean, yerr = std, fmt = fmt[masktype], capsize = 2, alpha = 0.5, color=colors[masktype], label = masktype)
            plt.ylabel('Test accuracy')
            plt.xlabel('Proportion of nodes/feature maps remaining')
            plt.legend(loc = 'lower left')
            plt.savefig(f'{experiment_name}/oracle_comparison_{name}.png')

        # Plot figures for early stopping iteration
        plt.figure()
        plt.gca().invert_xaxis()
        plt.xscale('log')
        plt.xticks([0.1,0.2,0.3,0.4,0.5, 0.6, 0.7, 0.8, 0.9, 1.0], ['0.1','0.2','0.3','0.4','0.5', '0.6', '0.7', '0.8', '0.9', '1.0'], rotation = 45)
        for masktype in masktypes:
            length = len(percent_removed[masktype])//num_trials
            mean = []
            std = []
            for i in range(length):
                mean.append(np.mean(early_stopping[masktype][i::length]))
                std.append(1.96*np.std(early_stopping[masktype][i::length])/np.sqrt(num_trials))
            plt.errorbar(percent_removed[masktype][:length], mean, yerr = std, fmt = fmt[masktype], capsize = 2, alpha = 0.5, color=colors[masktype], label = masktype)
            plt.ylabel('Early stopping iteration')
            plt.xlabel('Proportion of nodes/feature maps remaining')
            plt.legend(loc = 'lower left')
            plt.savefig(f'{experiment_name}/early_stopping_{name}.png')
        

## Section 1: Model A - Multilayer perceptron initialization for MNIST 


In [ ]:
class MlpModelA(NodePrune):
    """MNIST"""
    def __init__(self, number_nodes_layer_1=40, number_nodes_layer_2=40, dense=10):
        self.number_nodes_layer_1 = number_nodes_layer_1
        self.number_nodes_layer_2 = number_nodes_layer_2
        self.dense = dense
        self.x_train = None 
        self.x_val = None 
        self.y_train = None
        self.y_val = None
        self.x_test = None
        self.y_test = None
        self.get_dataset()
        
    def get_dataset(self):
        mnist = keras.datasets.mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_train = x_train/255.0
        x_test = x_test.reshape(-1, 28, 28, 1)
        x_test = x_test/255.0
        self.x_test = x_test
        self.y_test = y_test

        # split into train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_train, y_train, stratify = y_train, test_size=0.1, random_state=42)

    # We drop the nodes when creating a new model.
    def initialize_model(self, mask=None):
        """
        Initialize a multi-layer perceptron model. Droping or retention of nodes 
        happens when add_mask() is called.
        Args:
            mask: Boolean - if None, a mask dictionary is created, otherwise, mask
            has already been created
        Returns:
            mask:  mask: Dict - key is a layer, value is numpy array of mask
            model: a tf.keral functional Model made up of layers
            activation_array: array of activations (i.e output of Relu(Wx + b))
        """
        layer_index = 0
        layer_name = 'layer_'
        activation_array = []

        if mask is None:
            mask = {} 
            initialize = True
        else:
            initialize = False

        inputs = Input(shape = [28, 28, 1], name=layer_name+str(layer_index)) 
        layer_index += 1
        
        model = Flatten(name=layer_name+str(layer_index))(inputs)
        mask, model, activation_array, layer_index = self.add_mask(mask, model, layer_index, activation_array, initialize=initialize, in_out_layers=True)
        model = Dense(40, activation='relu', name=layer_name+str(layer_index))(model) 
        mask, model, activation_array, layer_index = self.add_mask(mask, model, layer_index, activation_array, initialize=initialize, in_out_layers=False)
        model = Dense(40, activation='relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_mask(mask, model, layer_index, activation_array, initialize=initialize, in_out_layers=False)
        out = Dense(10, activation='softmax', name=layer_name+str(layer_index))(model)
        out2 = Dense(10, activation='relu', name=layer_name+str(layer_index))(model) #the output layer is needed
        mask, model, activation_array, layer_index = self.add_mask(mask, out, layer_index, activation_array, initialize=initialize, in_out_layers=True) 
        #overide the last activation which is softmax with relu
        activation_array[-1] = Multiply()([out2, tf.multiply(tf.ones_like(out2), tf.ones_like(out2))])

        model = Model(inputs = [inputs], outputs = [out])
        activation_model = Model(inputs = [inputs], outputs = activation_array)
        return mask, model, activation_model


## Section 2: Model B - CNN initialization for MNIST

In [ ]:
class CnnModelB(NodePrune):
    """MNIST"""
    def __init__(self, number_filters_conv_1=64, number_filters_conv_2=64, dense=10):
        self.number_filters_conv_1 = number_filters_conv_1
        self.number_filters_conv_2 = number_filters_conv_2
        self.dense = dense
        self.x_train = None 
        self.x_val = None 
        self.y_train = None
        self.y_val = None
        self.x_test = None
        self.y_test = None
        self.get_dataset()

    def get_dataset(self):
        mnist = keras.datasets.mnist
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(-1, 28, 28, 1)
        x_train = x_train/255.0
        x_test = x_test.reshape(-1, 28, 28, 1)
        x_test = x_test/255.0
        self.x_test = x_test
        self.y_test = y_test

        # split into train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_train, y_train, stratify = y_train, test_size=0.1, random_state=42)

    def initialize_model(self, mask = None):
        """
        Initialize a cnn model. Droping or retention of filters 
        happens when add_filter_mask() is called.
        Args:
            mask: Boolean - if None, a mask dictionary is created, otherwise, mask
            has already been created
        Returns:
            mask:  mask: Dict - key is a layer, value is numpy array of mask
            model: a tf.keral functional Model made up of layers
            activation_array: array of activations (i.e output of Relu(Wx + b))
        """
        layer_index = 0
        layer_name = 'layer_'
        activation_array = []
        
        # if no mask specified, start with no mask
        if mask is None:
            mask = {}
            initialize = True
        else:
            initialize = False
        
        inputs = Input(shape = [28, 28, 1], name=layer_name+str(layer_index)) 
        layer_index += 1
        model = Conv2D(self.number_filters_conv_1, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(inputs)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize=initialize) 
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = Conv2D(self.number_filters_conv_2, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize=initialize)   
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1    
        model = Flatten(name=layer_name+str(layer_index))(model)
        layer_index += 1
        out = Dense(self.dense, activation = 'softmax', name=layer_name+str(layer_index))(model)
        activation_array.append(out)
        
        model = Model(inputs = [inputs], outputs = [out])
        activation_model = Model(inputs = [inputs], outputs = activation_array)
        return mask, model, activation_model

## Section 3: Model C - CNN initialization for CIFAR-10

In [ ]:
class CnnModelC(NodePrune):
    """CIFAR-10"""
    def __init__(self, number_filters_conv_1=64, number_filters_conv_2=64, number_filters_conv_3=128, number_filters_conv_4=128, dense_1=256, dense_2=10):
        self.number_filters_conv_1 = number_filters_conv_1
        self.number_filters_conv_2 = number_filters_conv_2
        self.number_filters_conv_3 = number_filters_conv_3
        self.number_filters_conv_4 = number_filters_conv_4
        self.dense_1 = dense_1
        self.dense_2 = dense_2
        self.x_train = None 
        self.x_val = None 
        self.y_train = None
        self.y_val = None
        self.x_test = None
        self.y_test = None
        self.get_dataset()

    def get_dataset(self):
        cifar = keras.datasets.cifar10
        (x_train, y_train), (x_test, y_test) = cifar.load_data()
        x_train = x_train.reshape(-1, 32, 32, 3)
        x_train = x_train/255.0
        x_test = x_test.reshape(-1, 32, 32, 3)
        x_test = x_test/255.0
        self.x_test = x_test
        self.y_test = y_test

        # split into train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_train, y_train, stratify = y_train, test_size=0.1, random_state=42)

    
    def initialize_model(self, mask = None):
        """
        Initialize a cnn model with a certain mask. Droping or retention of filters 
        happens when add_filter_mask() is called.
        Args:
            mask: Boolean - if None, a mask dictionary is created, otherwise, mask
            has already been created
        Returns:
            mask:  mask: Dict - key is a layer, value is numpy array of mask
            model: a tf.keral functional Model made up of layers
            activation_array: array of activations (i.e output of Relu(Wx + b))
        """
        layer_index = 0
        layer_name = 'layer_'
        activation_array = []
        
        # if no mask specified, start with no mask
        if mask is None:
            mask = {}
            initialize = True
        else:
            initialize = False
        
        inputs = Input(shape = [32, 32, 3], name=layer_name+str(layer_index))
        layer_index += 1

        ## Define your model architecture below
        ## For every Conv Layer, follow up with an add_filter_mask line to add the filter mask
        
        # Model C (Conv layers)
        model = Conv2D(self.number_filters_conv_1, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(inputs)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = Conv2D(self.number_filters_conv_2, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        model = Conv2D(self.number_filters_conv_3, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = Conv2D(self.number_filters_conv_4, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = Flatten(name=layer_name+str(layer_index))(model)
        layer_index += 1
        # Multiply by mask on nodes
        model = Dense(self.dense_1, activation = 'relu', name=layer_name+str(layer_index))(model)
        layer_index += 1
        out = Dense(self.dense_2, activation = 'softmax', name=layer_name+str(layer_index))(model)
        activation_array.append(out)
        
        model = Model(inputs = [inputs], outputs = [out])
        activation_model = Model(inputs = [inputs], outputs = activation_array)
        
        return mask, model, activation_model

## Section 4: ResNet18 initialization for CIFAR-10

In [ ]:
class ResNet18(NodePrune):
    """CIFAR-10"""
    def __init__(self):
        self.x_train = None 
        self.x_val = None 
        self.y_train = None
        self.y_val = None
        self.x_test = None
        self.y_test = None
        self.get_dataset()
    
    def get_dataset(self):
        cifar = keras.datasets.cifar10
        (x_train, y_train), (x_test, y_test) = cifar.load_data()
        x_train = x_train.reshape(-1, 32, 32, 3)
        x_train = x_train/255.0
        x_test = x_test.reshape(-1, 32, 32, 3)
        x_test = x_test/255.0
        self.x_test = x_test
        self.y_test = y_test

        # split into train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_train, y_train, stratify = y_train, test_size=0.1, random_state=42)

    def initialize_model(self, mask = None):
        """
        Initialize a ResNet-18 model with a certain model. Droping or retention of filters 
        happens when add_filter_mask() is called.
        Args:
            mask: Boolean - if None, a mask dictionary is created, otherwise, mask
            has already been created
        Returns:
            mask:  mask: Dict - key is a layer, value is numpy array of mask
            model: a tf.keral functional Model made up of layers
            activation_array: array of activations (i.e output of Relu(Wx + b))
        """
        layer_index = 0
        layer_name = 'layer_'
        activation_array = []

        # if no mask specified, start with no mask
        if mask is None:
            mask = {}
            initialize = True
        else:
            initialize = False
        
        inputs = Input(shape = [32, 32, 3], name=layer_name+str(layer_index))
        layer_index += 1

        # # [64] x 1, 1/2
        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(inputs)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)  
        shortcut = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        # [64, 64] x 2, 1/2
        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        shortcut = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1

        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1
        shortcut = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model) 
        layer_index += 1

        # [128, 128] x 2, 1/2
        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut) 
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model) 
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        shortcut = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut) 
        #model = BatchNormalization()(model)
        mask, shortcut, activation_array, layer_index = self.add_filter_mask(mask, shortcut, layer_index, activation_array, initialize = initialize)
        shortcut = Add(name=layer_name+str(layer_index))([shortcut, model]) 
        layer_index += 1

        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut) 
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1
        shortcut = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        # [256, 256] x 2, 1/2
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        shortcut = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, shortcut, activation_array, layer_index = self.add_filter_mask(mask, shortcut, layer_index, activation_array, initialize = initialize)
        shortcut = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1

        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1
        shortcut = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        # [512, 512] x 2, 1/2
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        shortcut = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, shortcut, activation_array, layer_index = self.add_filter_mask(mask, shortcut, layer_index, activation_array, initialize = initialize)
        shortcut = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1

        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(shortcut)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        #model = BatchNormalization()(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Add(name=layer_name+str(layer_index))([shortcut, model])
        layer_index += 1

        model = GlobalAveragePooling2D(name=layer_name+str(layer_index))(model)
        layer_index += 1  
        # Multiply by mask on nodes (Drop nodes)
        out = Dense(10, activation = 'softmax', name=layer_name+str(layer_index))(model)
        activation_array.append(out)
        
        model = Model(inputs = [inputs], outputs = [out])
        activation_model = Model(inputs = [inputs], outputs = activation_array)

        return mask, model, activation_model


## Section 5: VGG19 initialization for CIFAR-10

In [ ]:
class Vgg19(NodePrune):
    """CIFAR-10"""
    def __init__(self):
        self.x_train = None 
        self.x_val = None 
        self.y_train = None
        self.y_val = None
        self.x_test = None
        self.y_test = None
        self.get_dataset()
    
    def get_dataset(self):
        cifar = keras.datasets.cifar10
        (x_train, y_train), (x_test, y_test) = cifar.load_data()
        x_train = x_train.reshape(-1, 32, 32, 3)
        x_train = x_train/255.0
        x_test = x_test.reshape(-1, 32, 32, 3)
        x_test = x_test/255.0
        self.x_test = x_test
        self.y_test = y_test

        # split into train and validation sets
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(x_train, y_train, stratify = y_train, test_size=0.1, random_state=42)

    def initialize_model(self, mask = None):
        """
        Initialize a VGG-19 model with a certain mask. Droping or retention of filters 
        happens when add_filter_mask() is called.
        Args:
            mask: Boolean - if None, a mask dictionary is created, otherwise, mask
            has already been created
        Returns:
            mask:  mask: Dict - key is a layer, value is numpy array of mask
            model: a tf.keral functional Model made up of layers
            activation_array: array of activations (i.e output of Relu(Wx + b))
        """
        layer_index = 0
        layer_name = 'layer_'
        activation_array = []
        
        # if no mask specified, start with no mask
        layer = 0
        if mask is None:
            mask = {}
            initialize = True
        else:
            initialize = False
        
        inputs = Input(shape = [32, 32, 3], name=layer_name+str(layer_index))
        layer_index += 1

        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(inputs)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        model = Conv2D(64, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)   
        layer_index += 1
        model = BatchNormalization(name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(128, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        layer_index += 1
        model = BatchNormalization(name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(256, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        layer_index += 1
        model = BatchNormalization(name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1

        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        layer_index += 1
        model = BatchNormalization(name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        model = Conv2D(512, (3, 3), padding = 'same', activation = 'relu', name=layer_name+str(layer_index))(model)
        mask, model, activation_array, layer_index = self.add_filter_mask(mask, model, layer_index, activation_array, initialize = initialize)
        layer_index += 1
        model = BatchNormalization(name=layer_name+str(layer_index))(model)
        layer_index += 1
        model = MaxPooling2D((2, 2), name=layer_name+str(layer_index))(model)
        layer_index += 1
        
        model = Flatten(name=layer_name+str(layer_index))(model)
        layer_index += 1

        # Multiply by mask on nodes (Drop nodes)
        model = Dense(256, activation = 'relu', name=layer_name+str(layer_index))(model)
        layer_index += 1

        out = Dense(10, activation = 'softmax', name=layer_name+str(layer_index))(model)
        activation_array.append(out)
        
        model = Model(inputs = [inputs], outputs = [out])
        activation_model = Model(inputs = [inputs], outputs = activation_array)
        
        return mask, model, activation_model

## Section 6: MNIST Experiments

In [ ]:
class MlpMnistExperiment():
    """MLP MNIST Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using MLP of different node configurations
        Arguments:
            self: current object.
        Returns:
            None
        """
        
        print(f"Evaluating on MLP_40_40...")
        ## Change your folder name accordingly
        experiment_name = 'mlp_mnist_modelA_dense40_dense40'

        ## Model evaluation to compare all metrics 
        cache_of_mlp_model_40_40 = MlpModelA(40, 40).generate_model(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'mnist_dense40_dense40_evaluate'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_mlp_model_40_40, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_mlp_model_40_40, model_name)
        print(f"Cache saved as {model_name}.")
        
        ## Change your model name accordingly
        experiment_name = 'mlp_mnist_modelA_dense20_dense40'
        print(f"Evaluating on MLP_20_40...")
        cache_of_mlp_model_20_40 = MlpModelA(20, 40).generate_model(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        model_name = 'mnist_dense20_dense40_evaluate'
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_mlp_model_20_40, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_mlp_model_20_40, model_name)
        print(f"Cache saved as {model_name}.")
        
        ## Change your model name accordingly
        experiment_name = 'mlp_mnist_modelA_dense40_dense20'
        print(f"Evaluating on MLP_40_20...")
        cache_of_mlp_model_40_20 = MlpModelA(40, 20).generate_model(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        model_name = 'mnist_dense40_dense20_evaluate'
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_mlp_model_40_20, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_mlp_model_40_20, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnMnistExperimentModelB_64_64():
    """CNN MNIST Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using Model B
        Arguments:
            self: current object.
        Returns:
            None
        """
        
        print(f"Evaluating on Model B Conv64-Conv64...")
        ## Change your folder name accordingly
        experiment_name = 'cnn_mnist_modelB_conv64_conv64'

        ## Model evaluation to compare all metrics
        cache_of_cnn_modelB_conv64_conv64 = CnnModelB(64, 64).generate_model(experiment_name, num_epochs= 100, num_trials=2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'cnn_mninst_conv64_conv64_evaluate'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_modelB_conv64_conv64, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_modelB_conv64_conv64, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnMnistExperimentModelB_32_64():
    """CNN MNIST Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using Model B
        Arguments:
            self: current object
        Returns:
            None
        """
        
        ## Change your model name accordingly
        experiment_name = 'cnn_mnist_modelB_conv32_conv64'
        print(f"Evaluating on Model B Conv32-Conv64...")
        cache_of_cnn_modelB_conv32_conv64 = CnnModelB(32, 64).generate_model(experiment_name, num_epochs= 100, num_trials=2)
        print(f"Evaluation on done.")

        model_name = 'cnn_mninst_conv32_conv64_evaluate'
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_modelB_conv32_conv64, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_modelB_conv32_conv64, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnMnistExperimentModelB_64_32():
    """CNN MNIST Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using Model B
        Arguments:
            self: current object
        Returns:
            None
        """
        
        ## Change your model name accordingly
        experiment_name = 'cnn_mnist_modelB_conv64_conv32'
        print(f"Evaluating on Model B Conv64-Conv32...")
        cache_of_cnn_modelB_conv64_conv32 = CnnModelB(64, 32).generate_model(experiment_name, num_epochs= 100, num_trials=2)
        print(f"Evaluation on done.")

        model_name = 'cnn_mninst_conv64_conv32_evaluate'
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_modelB_conv64_conv32, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_modelB_conv64_conv32, model_name)

## Section 7: CIFAR-10 Experiments

In [ ]:
class CnnCifar10ExperimentModelC_64_64_128_128():
    """CNN CIFAR-10 Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using Model C
        Arguments:
            self: current object. 
        Returns:
            None
        """
        
        print(f"Evaluating on Model C Conv64-Conv64-Conv128-Conv128...")
        ## Change your folder name accordingly
        experiment_name = 'cnn_cifar10_modelC_conv64_conv64_conv128_conv128'

        ## Model evaluation to compare all metrics
        cache_of_cnn_modelC_conv64_conv64_conv128_conv128 = CnnModelC(64, 64, 128, 128).generate_model(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'cnn_cifar10_conv64_conv64_conv128_conv128_evaluate'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_modelC_conv64_conv64_conv128_conv128, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_modelC_conv64_conv64_conv128_conv128, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnCifar10ExperimentModelC_128_128_256_256():
    """CNN CIFAR-10 Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using Model C
        Arguments:
            self: current object. 
        Returns:
            None
        """
        
        print(f"Evaluating on Model C Conv128-Conv128-Conv256-Conv256...")        
        ## Change your model name accordingly
        experiment_name = 'cnn_cifar10_modelC_conv128_conv128_conv256_conv256'
        cache_of_cnn_modelC_conv128_conv128_conv256_conv256 = CnnModelC(128, 128, 256, 256).generate_model(experiment_name, num_epochs=100, num_trials=2)
        print(f"Evaluation on done.")

        model_name = 'cnn_cifar10_conv128_conv128_conv256_conv256_evaluate'
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_modelC_conv128_conv128_conv256_conv256, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_modelC_conv128_conv128_conv256_conv256, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnCifar10ResNet18Experiment():
    """CNN CIFAR-10 ResNet18 Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using MLP of different node configurations
        Arguments:
            print_progress: Boolean. If true, execution progress will be printed, if false, nothing will be printed.
        Returns:
            None
        """
        
        print(f"Evaluating on ResNet18...")
        ## Change your folder name accordingly
        experiment_name = 'cnn_cifar10_resnet18'

        ## Model evaluation to compare all metrics
        cache_of_cnn_cifar10_resnet18 = ResNet18().generate_model(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'cnn_cifar10_resnet18_evaluate'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_cifar10_resnet18, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_cifar10_resnet18, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class CnnCifar10Vgg19Experiment():
    """CNN CIFAR-10 VGG19 Experiments"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset using MLP of different node configurations
        Arguments:
            print_progress: Boolean. If true, execution progress will be printed, if false, nothing will be printed.
        Returns:
            None
        """
        
        print(f"Evaluating on VGG19...")
        ## Change your folder name accordingly
        experiment_name = 'cnn_cifar10_vgg19'

        ## Model evaluation to compare all metrics
        cache_of_cnn_cifar10_vgg19 = Vgg19().generate_model(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'cnn_cifar10_vgg19_evaluate'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_cnn_cifar10_vgg19, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_cnn_cifar10_vgg19, model_name)
        print(f"Cache saved as {model_name}.")

## Section 8: Oracle Comparison Experiment

In [ ]:
class OracleComparisonModelA_20_20():
    """Oracle MNIST Experiments on ModelA"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset on Model A
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on MLP_20_20...")
        ## Change your folder name accordingly
        experiment_name = 'oracle_mlp_mnist_modelA_dense20_dense20'

        ## Model evaluation to compare all metrics
        cache_of_oracle_mlp_model_20_20 = MlpModelA(20, 20).oracle_model(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'oracle_mlp_mnist_modelA_dense20_dense20'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_oracle_mlp_model_20_20, model_name, experiment_name, num_trials = 2, oracle = True)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_oracle_mlp_model_20_20, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class OracleComparisonModelB_20_20():
    """Oracle MNIST Experiments on ModelB"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset on Model B
        Arguments:
            self: current object.
        Returns:
            None
        """    
        print(f"Evaluating on Model B Conv20-Conv20...")
        ## Change your folder name accordingly
        experiment_name = 'oracle_cnn_mnist_modelB_conv20_conv20'

        ## Model evaluation to compare all metrics
        cache_of_oracle_cnn_mnist_modelB_conv20_conv20 = CnnModelB(20, 20).oracle_model(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'oracle_cnn_mninst_modelB_conv20_conv20'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_oracle_cnn_mnist_modelB_conv20_conv20, model_name, experiment_name, num_trials = 2, oracle = True)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_oracle_cnn_mnist_modelB_conv20_conv20, model_name)
        print(f"Cache saved as {model_name}.")

## Section 9: Random Initialization Experiment

In [ ]:
class RandomInitializationModelA_20_20():
    """Random Initialization on ModelA on MNIST"""
    def run(self):
        """
        Runs the experiments on the MNIST on Models A on MNIST
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on ModelA - MLP_20_20...")
        ## Change your folder name accordingly
        experiment_name = 'randominit_mlp_mnist_modelA_dense20_dense20'

        ## Model evaluation to compare all metrics
        cache_of_randominit_mlp_mnist_model_20_20 = MlpModelA(20, 20).compare_random(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'randominit_mlp_mnist_modelA_dense20_dense20'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_randominit_mlp_mnist_model_20_20, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_randominit_mlp_mnist_model_20_20, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class RandomInitializationModelB_64_64():
    """Random Initialization on ModelB on MNIST"""
    def run(self):
        """
        Runs the experiments on the MNIST on Model B
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on Model B Conv64-Conv64...")
        ## Change your folder name accordingly
        experiment_name = 'randominit_cnn_mnist_modelB_conv64_conv64'

        ## Model evaluation to compare all metrics
        cache_of_randominit_cnn_modelB_conv64_conv64 = CnnModelB(64, 64).compare_random(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'randominit_cnn_modelB_mninst_conv64_conv64'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_randominit_cnn_modelB_conv64_conv64, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_randominit_cnn_modelB_conv64_conv64, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class RandomInitializationModelC_64_64_128_128():
    """Random Initialization on ModelC on CIFAR-10"""
    def run(self):
        """
        Runs the experiments on the MNIST on CIFAR-10 on Model C
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on Model C Conv64-Conv64-Conv128-Conv128...")
        ## Change your folder name accordingly
        experiment_name = 'randominit_cnn_cifar10_modelC_conv64_conv64_conv128_conv128'

        ## Model evaluation to compare all metrics
        cache_of_randominit_cnn_modelC_conv64_conv64_conv128_conv128 = CnnModelC(64, 64, 128, 128).compare_random(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'randominit_cnn_cifar10_conv64_conv64_conv128_conv128'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_randominit_cnn_modelC_conv64_conv64_conv128_conv128, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_randominit_cnn_modelC_conv64_conv64_conv128_conv128, model_name)
        print(f"Cache saved as {model_name}.")

## Section 10: Percentage of nodes/filters to Drop Experiment

In [ ]:
class PercentageToDropModelA_20_20():
    """Random Initialization on Model A on MNIST"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset on MLP Model A 
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on ModelA - MLP_20_20...")
        ## Change your folder name accordingly
        experiment_name = 'percentage_mlp_mnist_modelA_dense20_dense20'

        ## Model evaluation to compare all metrics
        cache_of_percentage_mlp_mnist_model_20_20 = MlpModelA(20, 20).compare_percent_mask(experiment_name, num_epochs=100, num_trials = 2, cnn=False)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'percentage_mlp_mnist_modelA_dense20_dense20'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_percentage_mlp_mnist_model_20_20, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_percentage_mlp_mnist_model_20_20, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class PercentageToDropModelB_64_64():
    """Random Initialization on Model B on MNIST"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset on Model B
        Arguments:
            self: current object.
        Returns:
            None
        """        
        print(f"Evaluating on Model B Conv64-Conv64...")
        ## Change your folder name accordingly
        experiment_name = 'percentage_cnn_mnist_modelB_conv64_conv64'

        ## Model evaluation to compare all metrics
        cache_of_percentage_cnn_modelB_conv64_conv64 = CnnModelB(64, 64).compare_percent_mask(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'percentage_cnn_modelB_mninst_conv64_conv64'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_percentage_cnn_modelB_conv64_conv64, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_percentage_cnn_modelB_conv64_conv64, model_name)
        print(f"Cache saved as {model_name}.")

In [ ]:
class PercentageToDropModelC_64_64_128_128():
    """Random Initialization on ModelC on CIFAR-10"""
    def run(self):
        """
        Runs the experiments on the MNIST dataset on Model C
        Arguments:
            self: current object.
        Returns:
            None
        """
        print(f"Evaluating on Model C Conv64-Conv64-Conv128-Conv128...")
        ## Change your folder name accordingly
        experiment_name = 'percentage_cnn_cifar10_modelC_conv64_conv64_conv128_conv128'

        ## Model evaluation to compare all metrics
        cache_of_percentage_cnn_modelC_conv64_conv64_conv128_conv128 = CnnModelC(64, 64, 128, 128).compare_percent_mask(experiment_name, num_epochs=100, num_trials = 2)
        print(f"Evaluation on done.")

        ## Change your model name accordingly
        model_name = 'percentage_cnn_cifar10_conv64_conv64_conv128_conv128'
    
        ## If want to print oraclecomparison graph, set to oracle = True (only for oraclemodel)
        print(f"Printing graph ...")
        NodePrune().print_graph(cache_of_percentage_cnn_modelC_conv64_conv64_conv128_conv128, model_name, experiment_name, num_trials = 2, oracle = False)
        print(f"Graph printed. Saving cache ...")
        NodePrune().save_file(cache_of_percentage_cnn_modelC_conv64_conv64_conv128_conv128, model_name)
        print(f"Cache saved as {model_name}.")

## Section: Main - entry point of the code

In [ ]:
if __name__ == "__main__":
    """Experiments"""
    MlpMnistExperiment().run()
    #CnnMnistExperimentModelB_64_64().run()
    #CnnMnistExperimentModelB_32_64().run()
    #CnnMnistExperimentModelB_64_32().run()
    #CnnCifar10ExperimentModelC_64_64_128_128().run()
    #CnnCifar10ExperimentModelC_128_128_256_256().run()
    #CnnCifar10ResNet18Experiment().run()
    #CnnCifar10Vgg19Experiment().run()
    #OracleComparisonModelA_20_20().run()
    #OracleComparisonModelB_20_20().run()
    #RandomInitializationModelA_20_20().run()
    #RandomInitializationModelB_64_64().run()
    #RandomInitializationModelC_64_64_128_128().run()
    #PercentageToDropModelA_20_20().run()
    #PercentageToDropModelB_64_64().run()
    #PercentageToDropModelC_64_64_128_128().run()
